In [1]:
#Dependencies
from matplotlib import pyplot as plt
from scipy.stats import linregress
import numpy as np
from sklearn import datasets
import pandas as pd
import requests
import gmaps
import os
import json
import time

#Import API Key
from api_keys import g_key

In [2]:
#the set the path
csv_path = "../Resources/country_vaccinations_clean.csv"

# Read our COVID data into pandas
covid_df = pd.read_csv(csv_path)
covid_df

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name
0,Algeria,DZA,1/29/2021,0,0,0,0,0,0,0,0,0,Sputnik V,Ministry of Health
1,Algeria,DZA,1/30/2021,30,0,0,30,30,0,0,0,1,Sputnik V,Ministry of Health
2,Andorra,AND,1/25/2021,576,576,0,0,0,0,0,0,0,Pfizer/BioNTech,Government of Andorra
3,Andorra,AND,1/26/2021,0,0,0,0,66,0,0,0,854,Pfizer/BioNTech,Government of Andorra
4,Andorra,AND,1/27/2021,0,0,0,0,66,0,0,0,854,Pfizer/BioNTech,Government of Andorra
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463,Wales,0,2/2/2021,463657,462497,1160,22951,21530,14,14,0,6829,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom
2464,Wales,0,2/3/2021,491786,490570,1216,28129,22149,15,15,0,7025,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom
2465,Wales,0,2/4/2021,524677,523042,1635,32891,23101,16,16,0,7327,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom
2466,Wales,0,2/5/2021,559468,556997,2471,34791,25788,17,17,0,8179,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom


## Store Cleaned Data in DataFrame   

In [3]:
#Q1:  Total number of people fully vaccinated by country and ratios-daily
#DF:  Country, Code, Date, People Fully Vaccinated, People Fully Vac/Hundred, add vaccine and source name
#Dict: Total number of people fully vaccinated - this is the number of people that received the entire set of immunization 
#according to the immunization scheme (typically 2); at a certain moment in time, there might be a certain number of people that received one vaccine and another number (smaller) of people that received all vaccines in the scheme;
#Total number of people fully vaccinated per hundred - ratio (in percent) between population fully immunized and total population up to the date in the country;
#Show covid_count() as part of understanding the data structure.  Show count and describe.
#Graph Type:  Bar Chart


In [4]:
#Q2: Which vaccines and mixes are being distributed most /least. 
#DF: 
#Graph Type: Bar


In [5]:
#Q3:  
#DF
#Create Heat Map